In [ ]:
!pip install -q jiwer torch faster-whisper transformers torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from functools import partial
from faster_whisper import WhisperModel
import time
import torchaudio

In [ ]:
using_faster_whisper = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "openai/whisper-large"

In [ ]:
if using_faster_whisper:
    !ct2-transformers-converter --model {model_id} --output_dir model_vlad --copy_files tokenizer_config.json preprocessor_config.json --quantization float16
    model = WhisperModel("model_vlad", device=str(device), compute_type="float16")

else:
    model = WhisperForConditionalGeneration.from_pretrained(model_id).to(device)
    model.generate = partial(model.generate, language="ro", task="transcribe")
    processor = WhisperProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def transcribe(audio):
    if using_faster_whisper:
        segments, _ = model.transcribe(audio, beam_size=5, language="ro", condition_on_previous_text=False)

        transcription = ""
        for segment in segments:
            transcription += segment.text

    else:
        waveform, original_sample_rate = torchaudio.load(audio)

        target_sample_rate = 16000
        if original_sample_rate != target_sample_rate:
          waveform = torchaudio.transforms.Resample(original_sample_rate, target_sample_rate)(waveform)

        inputs = processor(waveform.numpy(), sampling_rate=16000, return_tensors="pt").input_features.to(device)
        predicted_ids = model.generate(inputs)
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription

In [ ]:
start_time = time.time()

text = transcribe("./Recording.wav")

end_time = time.time()
time_taken = end_time - start_time

print("Transcription:", text)
print("Time taken:", time_taken, "seconds")

Transcription:  Bună, mă numesc Sârbu Vlad și sunt student în anul 3 la Facultatea de Automatică și Calculatoare. Aș vrea să știu care sunt drepturile studentului.
Time taken: 3.67409086227417 seconds
